In [1]:
import sys
sys.path.append("..")

import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as ipw

from lib.dataset_wrapper import Dataset
from lib import abx_utils
from lib import notebooks

In [2]:
DATASET_NAME = "pb2007"

dataset = Dataset(DATASET_NAME)
assert dataset.has_palate
items_ema = dataset.get_items_data("ema")

AssertionError: 

In [3]:
LIPS_CONSONANTS = ["p", "b", "m"]
TONGUE_CONSONANTS = ["t", "d", "k", "g"]
TESTED_CONSONANTS = dataset.phones_infos["consonants"]
CONSONANTS_COIL = {
    "t": slice(2, 4),
    "d": slice(2, 4),
    "k": slice(4, 6),
    "g": slice(4, 6),
}

vowels = dataset.phones_infos["vowels"]
dataset_lab = {DATASET_NAME: dataset.lab}
palate = dataset.palate

AttributeError: 'Dataset' object has no attribute 'palate'

In [4]:
consonants_indexes = abx_utils.get_datasets_phones_indexes(
    dataset_lab, TESTED_CONSONANTS, vowels
)

In [5]:
def show_consonant(consonant):
    consonant_indexes = consonants_indexes[consonant]
    
    def show_occurence(i_index):
        index = consonant_indexes[i_index]
        print(index)
        item_ema = items_ema[index[1]]
        consonant_ema = item_ema[index[2]]
        
        consonant_ema_norm = consonant_ema - consonant_ema.mean(axis=0)
        lower_lip_ema = consonant_ema_norm[:, 8:10]
        upper_lip_ema = consonant_ema_norm[:, 10:12]
        tongue_tip_ema = consonant_ema_norm[:, 2:4]
        tongue_mid_ema = consonant_ema_norm[:, 4:6]
        
        nb_frames = len(consonant_ema)
        
        YLIM = 5
        
        plt.figure()
        
        ax = plt.subplot(421)
        ax.set_ylim(-YLIM, YLIM)
        ax.plot(upper_lip_ema[:, 0])
        ax = plt.subplot(423)
        ax.set_ylim(-YLIM, YLIM)
        ax.plot(upper_lip_ema[:, 1])
        ax = plt.subplot(425)
        ax.set_ylim(-YLIM, YLIM)
        ax.plot(lower_lip_ema[:, 0])
        ax = plt.subplot(427)
        ax.set_ylim(-YLIM, YLIM)
        ax.plot(lower_lip_ema[:, 1])
        
        upper_lip_y_speed = np.diff(upper_lip_ema[:, 1])
        lower_lip_y_speed = np.diff(lower_lip_ema[:, 1])
        lips_distance = np.sqrt(np.sum((lower_lip_ema - upper_lip_ema) ** 2, axis=1))
        
        test = np.zeros(nb_frames, dtype="bool")
        test[1:-1] = (lips_distance[1:-1] < lips_distance[:-2]) & (lips_distance[1:-1] < lips_distance[2:])
        print(np.arange(nb_frames)[test])
        
        ax = plt.subplot(422)
        ax.plot(upper_lip_y_speed)
        ax = plt.subplot(424)
        ax.plot(test)
        ax = plt.subplot(426)
        ax.plot(lips_distance)
        ax = plt.subplot(428)
        ax.plot(np.diff(lips_distance))
        
        plt.show()
        
        notebooks.show_ema(consonant_ema)
    
    ipw.interact(show_occurence, i_index=range(len(consonant_indexes)))

ipw.interactive(show_consonant, consonant=consonants_indexes.keys())

interactive(children=(Dropdown(description='consonant', options=('p', 'f', 'b', 'v', 'm', 't', 's', 'd', 'z', …